<a href="https://colab.research.google.com/github/iamharshvishwakarma/recipe-ai/blob/main/Recipe_suggestion_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import cv2
import numpy as np
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def preprocess_ingredients(ingredients):
    return ingredients.lower().replace(",", " ")

def recommend_recipes(user_ingredients, df, top_n=5):
    df['Processed_Ingredients'] = df['Cleaned-Ingredients'].apply(preprocess_ingredients)
    user_ingredients = preprocess_ingredients(user_ingredients)

    # TF-IDF Vectorization
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(df['Processed_Ingredients'])
    user_tfidf = vectorizer.transform([user_ingredients])

    # Compute Cosine Similarity
    similarity_scores = cosine_similarity(user_tfidf, tfidf_matrix).flatten()
    top_indices = similarity_scores.argsort()[-top_n:][::-1]

    # Retrieve and display top matching recipes
    recommendations = df.iloc[top_indices][['TranslatedRecipeName', 'TotalTimeInMins', 'Cuisine', 'TranslatedInstructions', 'image-url']]
    return recommendations

def detect_ingredients_from_image(image_path):
    model = MobileNetV2(weights='imagenet')
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    preds = model.predict(img_array)
    decoded_preds = decode_predictions(preds, top=5)[0]

    ingredients = [item[1] for item in decoded_preds]  # Extract top predicted ingredients
    return ", ".join(ingredients)

def capture_image():
    cam = cv2.VideoCapture(0)
    cv2.namedWindow("Capture Image")
    while True:
        ret, frame = cam.read()
        if not ret:
            print("Failed to grab frame")
            break
        cv2.imshow("Capture Image", frame)
        if cv2.waitKey(1) & 0xFF == ord('c'):
            image_path = "captured_image.jpg"
            cv2.imwrite(image_path, frame)
            print("Image captured and saved as", image_path)
            break
    cam.release()
    cv2.destroyAllWindows()
    return image_path

# Load the dataset
df = pd.read_csv("/content/Cleaned_Indian_Food_Dataset.csv")

# User choice for input method
choice = input("Enter 'text' to input ingredients, 'image' to upload an image, or 'camera' to capture an image: ").strip().lower()

if choice == 'text':
    user_ingredients = input("Enter available ingredients (comma-separated): ")
elif choice == 'image':
    image_path = input("Enter the path of the ingredient image: ")
    user_ingredients = detect_ingredients_from_image(image_path)
    print(f"Detected ingredients: {user_ingredients}")
elif choice == 'camera':
    image_path = capture_image()
    user_ingredients = detect_ingredients_from_image(image_path)
    print(f"Detected ingredients: {user_ingredients}")
else:
    print("Invalid choice. Please enter 'text', 'image', or 'camera'.")
    exit()

recommended_recipes = recommend_recipes(user_ingredients, df)
print(recommended_recipes)


Enter 'text' to input ingredients, 'image' to upload an image, or 'camera' to capture an image: text
Enter available ingredients (comma-separated): water
                                   TranslatedRecipeName  TotalTimeInMins  \
2471                    Cucumber Ginger Lemonade Recipe               15   
1545  Mor Milagai Recipe - South Indian Sun Dried Ch...               10   
4326         Ujju Rotti Recipe (Indian Rice Flat Bread)               50   
5867                  Refreshing Sweet Lime Soda Recipe               15   
392   Mishti Doi Recipe - Traditional Bengali Sweet ...               40   

                   Cuisine                             TranslatedInstructions  \
2471           Continental  To start making Cucumber Ginger Mint Lemonade,...   
1545  South Indian Recipes  To begin making the Mor Milagai, first wash th...   
4326                Indian  To prepare Ujju Rotti Recipe (Indian Rice Flat...   
5867                Indian  To begin making the Refreshing Sweet 